<a href="https://www.kaggle.com/code/gpreda/simple-sequential-chain-with-llama-2-and-langchain?scriptVersionId=145694025" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction


## Objective  

Use Llama 2 and Langchain to create a multi-step task chain. We will compare the performance of two models, both from Meta.

## Models details  

* **Model #1**: Llama 2  
* **Variation**: 7b-chat-hf    
* **Version**: V1  
* **Framework**: PyTorch  


* **Model #2**: Llama 2  
* **Variation**: 13b-chat-hf    
* **Version**: V1  
* **Framework**: PyTorch  



LlaMA 2 model is pretrained and fine-tuned with 2 Trillion tokens and 7 to 70 Billion parameters which makes it one of the powerful open source models. It is a highly improvement over LlaMA 1 model. The two models selected here are those with 7 Billion and with 13 Billion parameters. 

# InstalIing, imports, utils

Install packages.

In [2]:
!pip install transformers accelerate einops langchain xformers bitsandbytes chromadb sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.6/211.6 MB 4.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.8/437.8 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 37.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 3.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 8.1 MB/s eta 0:00:00:00:0100:01
     ━━━

Import packages.

In [3]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time

from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain import PromptTemplate


## Initialize model, tokenizer, query pipeline  

Define the model, the device, and the bitsandbytes configuration.

In [4]:
model_1_id = '/kaggle/input/llama-2/pytorch/7b-chat-hf/1'

model_2_id = '/kaggle/input/llama-2/pytorch/13b-chat-hf/1'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

Prepare the model and the tokenizer.  

We perform this operation for both models (7b & 13b).

In [5]:
time_1 = time()
model_1_config = transformers.AutoConfig.from_pretrained(
    model_1_id,
)
model_1 = transformers.AutoModelForCausalLM.from_pretrained(
    model_1_id,
    trust_remote_code=True,
    config=model_1_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer_1 = AutoTokenizer.from_pretrained(model_1_id)
time_2 = time()
print(f"Prepare model #1, tokenizer: {round(time_2-time_1, 3)} sec.")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


Prepare model #1, tokenizer: 178.108 sec.


In [5]:
time_1 = time()
model_2_config = transformers.AutoConfig.from_pretrained(
    model_2_id,
)
model_2 = transformers.AutoModelForCausalLM.from_pretrained(
    model_2_id,
    trust_remote_code=True,
    config=model_2_config,
    quantization_config=bnb_config,
    use_safetensors=True,
    device_map='auto',
)
tokenizer_2 = AutoTokenizer.from_pretrained(model_2_id)
time_2 = time()
print(f"Prepare model #2, tokenizer: {round(time_2-time_1, 3)} sec.")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ImportError: Using `load_in_8bit=True` requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes `pip install -i https://test.pypi.org/simple/ bitsandbytes` or pip install bitsandbytes` 

Define a pipeline for each of the models, #1 (7b) and #2 (13b).

In [9]:
time_1 = time()
query_pipeline_1 = transformers.pipeline(
        "text-generation",
        model=model_1,
        tokenizer=tokenizer_1,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline #1: {round(time_2-time_1, 3)} sec.")

llm_1 = HuggingFacePipeline(pipeline=query_pipeline_1)


Prepare pipeline #1: 2.125 sec.


In [34]:
time_1 = time()
query_pipeline_2 = transformers.pipeline(
        "text-generation",
        model=model_2,
        tokenizer=tokenizer_2,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline #2: {round(time_2-time_1, 3)} sec.")

llm_2 = HuggingFacePipeline(pipeline=query_pipeline_2)

Prepare pipeline #2: 0.003 sec.



We test it by running a simple query. First with the 7b model (model #1) and then with the 13b model (model #2).

In [11]:
# checking model #1
llm_1(prompt="What is the most popular city in France for tourists? Just return the name of the city.")

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


' Unterscheidung between "Paris" and "Paris, France" is not necessary, as "Paris" already implies the country.\n\nAnswer:\nThe most popular city in France for tourists is Paris.'

In [35]:
# checking model #2
llm_2(prompt="What is the most popular city in France for tourists?")

'$\\ Chineseaddatăensisiring Humanloading'

# Define and execute the sequential chain


We define a chain with two tasks in sequence.  
The input for the second step is the output of the first step.  

In [19]:
def sequential_chain(country, llm):
    """
    Args:
        country: country selected
    Returns:
        None
    """
    time_1 = time()
    template = "What is the most popular city in {country} for tourists? Just return the name of the city."

    #  first task in chain
    first_prompt = PromptTemplate(

    input_variables=["country"],

    template=template)

    chain_one = LLMChain(llm = llm, prompt = first_prompt)

    # second step in chain
    second_prompt = PromptTemplate(

    input_variables=["city"],

    template="What are the top three things to do in this: {city} for tourists. Just return the answer as three bullet points.",)

    chain_two = LLMChain(llm=llm, prompt=second_prompt)

    # combine the two steps and run the chain sequence
    overall_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)
    overall_chain.run(country)
    time_2 = time()
    print(f"Run sequential chain: {round(time_2-time_1, 3)} sec.")

Test the sequence with Llama v2 **7b** chat HF model.

In [ ]:
final_answer = sequential_chain("France", llm_1)

Test the sequence with Llama v2 **13b** chat HF model.

In [20]:
final_answer = sequential_chain("France", llm_2)



> Entering new SimpleSequentialChain chain...


RuntimeError: mat1 and mat2 shapes cannot be multiplied (20x5120 and 1x2560)

Test the sequence with Llama v2 **7b** chat HF model.

In [ ]:
final_answer = sequential_chain("Germany", llm_1)

Test the sequence with Llama v2 **13b** chat HF model.

In [ ]:
final_answer = sequential_chain("Germany", llm_2)